In [43]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import gzip
from bed_manipulate import bed_intersection, bed_w_NoIntersection, bed_closest

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [44]:
DATADIR='/users/mtaranov/3D_fromATAC/data/'
PromoterCaptureFile=DATADIR+'PromoterCapture_Digest_Human_HindIII_baits_ID.bed'
ATACPeakFile=DATADIR+'SORTED_primary_keratinocyte-d00.GGR.Stanford_Greenleaf.ATAC-seq.b1.trim.PE2SE.nodup.tn5_pooled.pf.pval0.1.500000.naive_overlap.narrowPeak.gz'
ATACPeakIDRFile=DATADIR+'primary_keratinocyte-d00.GGR.Stanford_Greenleaf.ATAC-seq_rep1-rep2.IDR0.1.filt.narrowPeak.gz'
OutputFile=DATADIR+'out.bed'
InteractionsFileCaptureC='/mnt/lab_data/kundaje/mtaranov/3D/CaptureC_SC_bait-bait.bed.gz'
InteractionsFile_D0='/mnt/lab_data/kundaje/mtaranov/ChicagoCalls/superConfident_bait_bait_fromAdam/humKer_Sub_D2D8_merge.bed'
#InteractionsFile_D0='/mnt/lab_data/kundaje/mtaranov/ChicagoCalls/superConfident_bait_bait_fromAdam/test.bed'
site1=DATADIR+'site1'
site2=DATADIR+'site2'
outFile=DATADIR+'CaptureC_D0_bait-bait.bed'

In [47]:
get_bait_bait_contacts(InteractionsFile_D0, PromoterCaptureFile, )

In [46]:
from pybedtools import BedTool
import csv

#pulls bait-bait contacts from all-all interaction
def get_bait_bait_contacts(InteractionsFile_D0, PromoterCaptureFile):
    data = BedTool(InteractionsFile_D0)
    BedTool([i[0], i[1], i[2], i[6]] for i in data).saveas(site1)
    BedTool([i[3], i[4], i[5], i[6]] for i in data).saveas(site2)
    bed1_site1=BedTool(site1)
    bed1_site2=BedTool(site2)
    bed2 = BedTool(PromoterCaptureFile)

    with open(outFile, 'w') as outcsv:
        #configure writer to write standart csv file
            writer = csv.writer(outcsv, delimiter='\t', quotechar='|', quoting=csv.QUOTE_MINIMAL, lineterminator='\n')
            for interval in zip(bed1_site1.intersect(bed2, wao=True), bed1_site2.intersect(bed2, wao=True)):
                if interval[0][0] == interval[1][0]: # intra-chr contacts only 
                    if (int(interval[0][9]) != 0 and int(interval[1][9]) != 0):
                        writer.writerow([interval[0][0], interval[0][1], interval[0][2], interval[1][0], interval[1][1], interval[1][2], interval[1][3], interval[0][7], interval[1][7]])